### Monday, January 1, 2023

[microsoft/Orca-2-13b](https://huggingface.co/microsoft/Orca-2-13b)

docker container start hfpt_Dec14

This all runs.

In [1]:
!ls /root/.cache/huggingface/hub

models--TheBloke--Llama-2-13b-Chat-GPTQ
models--microsoft--Orca-2-13b
models--mistralai--Mistral-7B-Instruct-v0.2
version.txt


In [2]:
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--microsoft--Orca-2-13b /home/rob/Data3/huggingface/transformers
# Successfully copied 52.1GB to /home/rob/Data3/huggingface/transformers

In [3]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [4]:
import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_device("cuda")
else:
    torch.set_default_device("cpu")
  

In [5]:
modelName = "microsoft/Orca-2-13b"

In [6]:
# WOW! This actually loads!!
model = transformers.AutoModelForCausalLM.from_pretrained(modelName, load_in_8bit=True, torch_dtype=torch.float16)

# 43.8s
# 13848 MiB VRAM

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
# model = transformers.AutoModelForCausalLM.from_pretrained(modelName, device_map='auto')


In [8]:
# https://github.com/huggingface/transformers/issues/27132
# please use the slow tokenizer since fast and slow tokenizer produces different tokens
tokenizer = transformers.AutoTokenizer.from_pretrained(
        modelName,
        use_fast=False,
    )

In [9]:
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"


In [10]:
inputs = tokenizer(prompt, return_tensors='pt')

In [11]:
output_ids = model.generate(inputs["input_ids"],)

# 1m 47.6s

In [12]:
answer = tokenizer.batch_decode(output_ids)[0]

In [13]:
print(answer)

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to determine if a restaurant is popular among locals or mainly attracts tourists, such as:

- Looking at online reviews and ratings from different sources, such as TripAdvisor, Yelp, Google, etc. If a restaurant has mostly positive reviews from locals and a high rating, it might indicate that it is popular among locals. However, if a restaurant has mostly negative reviews from locals and a low rating, it might indicate that it is not popular among locals. If a restaurant has mostly positive reviews from tourists and a high ratin

In [14]:
# This example continues showing how to add a second turn message by the user to the conversation
second_turn_user_message = "Give me a list of the key points of your first answer."

# we set add_special_tokens=False because we dont want to automatically add a bos_token between messages
second_turn_message_in_markup = f"\n<|im_start|>user\n{second_turn_user_message}<|im_end|>\n<|im_start|>assistant"
second_turn_tokens = tokenizer(second_turn_message_in_markup, return_tensors='pt', add_special_tokens=False)
second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

output_ids_2 = model.generate(second_turn_input,)
second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

print(second_turn_answer)

# 21.0s

<s>  <|im_start|> system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. <|im_end|> 
 <|im_start|> user
How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful? <|im_end|> 
 <|im_start|> assistant
There are different ways to determine if a restaurant is popular among locals or mainly attracts tourists, such as:

- Looking at online reviews and ratings from different sources, such as TripAdvisor, Yelp, Google, etc. If a restaurant has mostly positive reviews from locals and a high rating, it might indicate that it is popular among locals. However, if a restaurant has mostly negative reviews from locals and a low rating, it might indicate that it is not popular among locals. If a restaurant has mostly positive reviews from tourists and a high ratin